In [1]:
# import dependencies
import os
import pandas as pd
import numpy as np
# import pymongo
# from pymongo import MongoClient
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
import warnings
warnings.filterwarnings('ignore')


In [2]:
# read in hurricane data
hurricane_data_df = pd.read_csv("static/data/atlantic.csv", encoding='utf-8')
# hurricane_data_df.head()

Ocean Temps data info:
Units: Degrees Celsius
Base Period: 1901-2000

In [3]:
# read in ocean temps data (nothing to do here)
oceantemps_data_df = pd.read_csv("static/data/oceantemps.csv", encoding='utf-8')
oceantemps_data_df["Year"] = oceantemps_data_df["Year"].astype(str)
oceantemps_data_df["Value"] = oceantemps_data_df["Value"].astype(str)
oceantemps_data_df.head()

,Year,Value
0,1880,-0.11
1,1881,-0.16
2,1882,-0.1
3,1883,-0.18
4,1884,-0.29


In [4]:
# read in annual CO2 data (Justan's - has more data points for his graph)
complete_co2_data_df = pd.read_csv("static/assets/data/c02.csv", encoding='utf-8')
complete_co2_data_df["year"] = complete_co2_data_df["year"].astype(str)
# complete_co2_data_df.head()

In [5]:
# reduce data frame to just have year and mean co2 for each year
complete_co2 = complete_co2_data_df[['year', 'monthly_average']]
meancomplete_co2 = complete_co2.groupby('year').mean()
# meancomplete_co2.head()

In [6]:
# #  rename dataframe to use for calculating percentage change year/year of co2
# perchange_co2 = meancomplete_co2

In [7]:
# round to 2 decimal places and reset index
meancomplete_co2 = meancomplete_co2.round(2)
meancomplete_co2_df = meancomplete_co2.reset_index()
# meancomplete_co2_df.head()

In [8]:
# rename the columns and this is final dataframe for year/mean co2 levels
meancomplete_co2_df.rename(columns={'year': 'year', 'monthly_average': 'meanco2'}, inplace=True)
meancomplete_co2_df["year"] = meancomplete_co2_df["year"].astype(str)
meancomplete_co2_df["meanco2"] = meancomplete_co2_df["meanco2"].astype(str)
meancomplete_co2_df.head()

,year,meanco2
0,1958,315.23
1,1959,315.98
2,1960,316.91
3,1961,317.64
4,1962,318.45


In [9]:
# use perchange_co2 = meancomplete_co2 to calculate year/year percentage change of co2 data
# perchange_co2.head()

In [10]:
# # calculate percentage change year/year of co2 data/round/ and reset index
# perchange_co2_df = perchange_co2.pct_change()
# perchange_co2_df = perchange_co2_df.round(3)
# perchange_co2_df = perchange_co2_df.reset_index()
# perchange_co2_df.tail()

In [11]:
# # rename the columns and this is final dataframe for year/year percentage change co2 levels
# perchange_co2_df.rename(columns={'year': 'year', 'monthly_average': 'co2change'}, inplace=True)
# perchange_co2_df["year"] = perchange_co2_df["year"].astype(str)
# perchange_co2_df.head()

In [12]:
# print(perchange_co2_df.dtypes)

In [13]:
# perchange_co2_df.to_csv("static/data/perchange_co2.csv", encoding='utf-8')

In [14]:
# read in annual co2 data (don't use this csv!!!)
# mean_co2_data_df = pd.read_csv("static/data/co2_annmean_mlo.csv", encoding='utf-8')
# mean_co2_data_df.head(5)


In [15]:
# rename columns - wont' let you use column names - "not in index"
#yearmean_co2 = mean_co2_data_df[['year', 'micromol_mol_mean']]
#yearmean_co2.head()

In [16]:
# reduce columns for hurricane data to what we need 
cropHurr = hurricane_data_df[["ID", "Name", "Date", "Time", "Event", "Status", "Latitude", "Longitude", "Maximum Wind"]]
# cropHurr.head()

In [17]:
# to calculate percentage change for storms year/year
# groupby ID to explore unique ID/status data - some storms never become hurricanes
cropHurrgroup = cropHurr.groupby(['ID', 'Status']).mean()
# cropHurrgroup.tail(10)

In [18]:
# find the last id's used in the data to isolate unique storms
cropHurrlast = cropHurr.groupby('ID').last()
# cropHurrlast.tail()

In [19]:
# drop last 4 characters for 'Date' to isolate year (not include month/day)
# reset index
cropHurrlast["Date"] = cropHurrlast["Date"].astype(str).str[:-4]
cropHurrlast = cropHurrlast.reset_index()
# cropHurrlast.tail()

In [20]:
# group by 'Date' to get counts of storms for each year
# reset index
cropHurrlastgroup = cropHurrlast.groupby('Date').count()
# cropHurrlastgroup.tail()

In [21]:
# reduce columns for just year and ID (storm counts/year)(don't reset index yet - do % change first)
hurrCount = cropHurrlastgroup[['ID']]
hurrCount = hurrCount.reset_index()
hurrCount.tail()

,Date,ID
160,2011,20
161,2012,19
162,2013,15
163,2014,9
164,2015,12


In [22]:
# # perform percentage change function to calculate changes from year to year (round to 3 decimal places)
# # reset index
# stormpercent_change_df = cropHurrlastgroup_df.pct_change()
# stormpercent_change_df = stormpercent_change_df.round(3)
# stormpercent_change_df = stormpercent_change_df.reset_index()
# stormpercent_change_df.tail()

In [23]:
# # rename columns for % change of storms for each year
# stormpercent_change_df.rename(columns={'Date': 'year', 'ID': 'stormchange'}, inplace=True)
# stormpercent_change_df.head()

In [24]:
# print(stormpercent_change_df.dtypes)

In [25]:
# stormpercent_change_df.to_csv("static/data/stormpercent_change.csv", encoding='utf-8')

In [26]:
# # check datatypes for hurricane data
# print(cropHurr.dtypes)

In [27]:
# remove last characters from latitude 'N' and longitude 'W' columns
cropHurr["Latitude"] = cropHurr["Latitude"].astype(str).str[:-1]
cropHurr["Longitude"] = cropHurr["Longitude"].astype(str).str[:-1]
# cropHurr.head()

In [28]:
# make W - longitude values negative 
cropHurr.Longitude = cropHurr.Longitude.astype(float)*(-1)

In [29]:
#MAIN HURRICANE DATASET TO USE
#TO DO:  reformat date and add year column
cleanHurr = cropHurr
# cleanHurr = cleanHurr["Date"].dt.strftime("%m/%d/%y")
cleanHurr['Date'] = pd.to_datetime(cleanHurr['Date'].astype('str'), format = '%Y/%m/%d')
cleanHurr["Year"] = cleanHurr["Date"].astype(str).str[:-6]
cleanHurr.to_csv("static/data/cleanHurr.csv", encoding='utf-8')
cleanHurr.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Year
0,AL011851,UNNAMED,1851-06-25,0,,HU,28.0,-94.8,80,1851
1,AL011851,UNNAMED,1851-06-25,600,,HU,28.0,-95.4,80,1851
2,AL011851,UNNAMED,1851-06-25,1200,,HU,28.0,-96.0,80,1851
3,AL011851,UNNAMED,1851-06-25,1800,,HU,28.1,-96.5,80,1851
4,AL011851,UNNAMED,1851-06-25,2100,L,HU,28.2,-96.8,80,1851


In [30]:
#TO DO: find max wind speed per unique storm ID
maxWind = cleanHurr.groupby("ID").max()
maxWind.to_csv("static/data/maxWind.csv", encoding='utf-8')
maxWind.tail()

,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Year
ID,,,,,,,,,
AL282005,GAMMA,2005-11-22,1800,,WV,17.1,-62.1,45,2005
AL291969,MARTHA,1969-11-25,1800,,TS,9.8,-81.0,80,1969
AL292005,DELTA,2005-11-29,1800,,TS,35.3,-1.0,60,2005
AL302005,EPSILON,2005-12-09,1800,,TS,34.5,-33.7,75,2005
AL312005,ZETA,2006-01-07,1800,,TS,26.3,-35.6,55,2006


In [31]:
#TO DO: Landfall dataset
landfall = cleanHurr.loc[cleanHurr["Event"] == " L"]
landfall.to_csv("static/data/landfall.csv", encoding='utf-8')
landfall.head()

,ID,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Year
4,AL011851,UNNAMED,1851-06-25,2100,L,HU,28.2,-96.8,80,1851
48,AL041851,UNNAMED,1851-08-23,2100,L,HU,30.1,-85.7,100,1851
96,AL061851,UNNAMED,1851-10-19,1500,L,TS,41.1,-71.7,50,1851
127,AL011852,UNNAMED,1852-08-26,600,L,HU,30.2,-88.6,100,1852
163,AL031852,UNNAMED,1852-09-12,0,L,HU,28.0,-82.8,70,1852


Create database

In [32]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float

In [33]:
# class OceanTemp(Base):
#     __tablename__ = 'ocean_temps'
#     Year = Column(String(255), primary_key=True)
#     Value = Column(String(255))

In [34]:
# Create an engine for the sqlite database
engine = create_engine("sqlite:///climateDB.sqlite", echo=False)
con = engine.connect()

In [35]:
cleanHurr.to_sql(name='hurricanes', con=engine, if_exists='replace', index=False)
hurrCount.to_sql(name='hurrCount', con=engine, if_exists='replace', index=False)
oceantemps_data_df.to_sql(name='ocean_temps', con=engine, if_exists='replace', index=False)
maxWind.to_sql(name='maxWind', con=engine, if_exists='replace', index=False)
meancomplete_co2_df.to_sql(name='meancomplete_co2_df', con=engine, if_exists='replace', index=False)
landfall.to_sql(name='landfall', con=engine, if_exists='replace', index=False)

In [36]:
pd.read_sql('select * from maxWind', con)

,Name,Date,Time,Event,Status,Latitude,Longitude,Maximum Wind,Year
0,UNNAMED,1851-06-28 00:00:00.000000,2100,L,TS,31.0,-94.8,80,1851
1,UNNAMED,1852-08-30 00:00:00.000000,1800,L,TS,41.0,-67.1,100,1852
2,UNNAMED,1853-08-05 00:00:00.000000,1200,,TS,32.5,-69.0,50,1853
3,UNNAMED,1854-06-27 00:00:00.000000,1800,L,TS,28.0,-92.5,70,1854
4,UNNAMED,1855-08-06 00:00:00.000000,1200,,HU,22.2,-98.0,90,1855
...,...,...,...,...,...,...,...,...,...
1809,GAMMA,2005-11-22 00:00:00.000000,1800,,WV,17.1,-62.1,45,2005
1810,MARTHA,1969-11-25 00:00:00.000000,1800,,TS,9.8,-81.0,80,1969
1811,DELTA,2005-11-29 00:00:00.000000,1800,,TS,35.3,-1.0,60,2005
1812,EPSILON,2005-12-09 00:00:00.000000,1800,,TS,34.5,-33.7,75,2005


In [37]:
# Create a database session object
session = Session(engine)

In [38]:
inspector = inspect(engine)
inspector.get_table_names()

['hurrCount',
 'hurricanes',
 'landfall',
 'maxWind',
 'meancomplete_co2_df',
 'ocean_temps']

In [39]:
columns = inspector.get_columns('ocean_temps')
for c in columns:
    print(c["name"], c["type"])

Year TEXT
Value TEXT


DO NOT USE AFTER THIS...OLD MONGODB CODE

In [40]:
# #Define connection link an instantiate client
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

# #Define hurricane database in Mongo
# #Define hurricane database in Mongo
# db = client.projectDB

# hurricanes = db.hurricanes

# oceanTemps = db.ocean

# carbon = db.carbon

# perchange_co2 = db.perchange_co2

# meancomplete_co2 = db.meancomplete_co2

# stormpercent_change = db.stormpercent_change

In [41]:
# #Instantiate for loop for populating hurricane collection
# for index, row in cleanHurr.iterrows():
#     #Populate post dictionary with information to be inserted into business collection
#     post = {
#         "ID": str(row["ID"]),
#         "Name": str(row["Name"]).strip(),
#         "Date": str(row["Date"]),
#         "Time": str(row["Time"]),
#         "Event": str(row["Event"]).strip(),
#         "Status": str(row["Status"]).strip(),
#         "Latitude": str(row["Latitude"]),
#         "Longitude": str(row["Longitude"]),
#         "Maximum Wind": str(row["Maximum Wind"])
#     }
#     #Insert data into hurricanes collection, post{} by post{}
#     hurricanes.insert_one(post)

In [42]:
# #Instantiate for loop for populating ocean temps collection
# for index, row in oceantemps_data_df.iterrows():
#         post = {
#             "Year": str(row["Year"]),
#             "Value": str(row["Value"])
#         }
#         #Insert data into oceanTemps collection, post{} by post{}
#         oceanTemps.insert_one(post)

In [43]:
# #Instantiate for loop for populating carbon data collection
# for index, row in complete_co2_data_df.iterrows():
#         post = {
#             "Year": str(row["year"]),
#             "Month": str(row["month"]),
#             "Date": str(row["decimal_date"]),
#             "Monthly Avg": str(row["monthly_average"])
#         }
#         #Insert data into carbon collection, post{} by post{}
#         carbon.insert_one(post)

In [44]:
# #Instantiate for loop for populating % change co2 collection
# for index, row in perchange_co2_df.iterrows():
#         post = {
#             "Year": str(row["year"]),
#             "co2 Change": str(row["co2change"])
#         }
#         #Insert data into % change co2 collection, post{} by post{}
#         perchange_co2.insert_one(post)

In [45]:
# #Instantiate for loop for populating mean complete co2 collection
# for index, row in meancomplete_co2_df.iterrows():
#         post = {
#             "Year": str(row["year"]),
#             "Mean co2": str(row["meanco2"])
#         }
#         #Insert data into mean complete co2 collection, post{} by post{}
#         meancomplete_co2.insert_one(post)

In [46]:
# #Instantiate for loop for populating storm % change collection
# for index, row in stormpercent_change_df.iterrows():
#         post = {
#             "Year": str(row["year"]),
#             "Stormchange": str(row["stormchange"])
#         }
#         #Insert data into storm % change collection, post{} by post{}
#         stormpercent_change.insert_one(post)